In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [2]:
with open('../Documents/Silica Water book.md', 'r') as file:
    markdown_string = file.read()
print(len(markdown_string))

427828


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_text(markdown_string)

len(all_splits)

599

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

doc_embeddings = embedder.embed_documents(all_splits)
print(f"Query embedding length: {len(doc_embeddings)}")
print(doc_embeddings[:5])

Query embedding length: 599
[[0.000614992983173579, 0.006393090356141329, 0.03398759290575981, 0.0679275318980217, 0.032804664224386215, 0.023307684808969498, 0.012447945773601532, -0.018140973523259163, -0.06269854307174683, -0.03041353076696396, -0.0033231116831302643, 0.010908471420407295, 0.006969561334699392, 0.006544051691889763, -0.059651728719472885, 0.04711555689573288, -0.09277717769145966, -0.023485329002141953, -0.02544255368411541, 0.01490108948200941, 0.05382445082068443, 0.05005144700407982, 0.025735344737768173, 0.00645383819937706, -0.012926983647048473, 0.11399441957473755, -0.022227726876735687, -0.06282665580511093, -0.00012629108096007258, 0.002703408943489194, 0.039072323590517044, 0.1044088751077652, -0.0017390649300068617, -0.10642530024051666, 0.045654021203517914, -0.007105608470737934, -0.1089354008436203, -0.036815546452999115, -0.09916560351848602, -0.030929235741496086, 0.030825629830360413, -0.08704444020986557, 0.028106404468417168, 0.04400764778256416, 

In [34]:
from langchain_core.documents import Document

documents: list[Document] = []
for i in range(len(all_splits)):
    document = Document(
        id = f"{i}",
        page_content = all_splits[i],
        metadata = {"source":"Book"}
    )
    documents.append(document)

print(documents[:5])

[Document(id='0', metadata={'source': 'Book'}, page_content='```\nSilica Water the Secret of Healthy Longevity in the\nAluminum Age\n```\nAuthor: Dr. Dennis N. Crouse, BSc Biochemistry, - Harvard College, Ph.D. Organic Chemistry\n\n- Harvard University Chemistry Department, Post-graduate courses: Understanding Dementia -\nWicking Faculty of Health, University of Tasmania, Fundamentals of Neuroscience - Harvard.'), Document(id='1', metadata={'source': 'Book'}, page_content='- Harvard University Chemistry Department, Post-graduate courses: Understanding Dementia -\nWicking Faculty of Health, University of Tasmania, Fundamentals of Neuroscience - Harvard.\n\n\n```\nCopyright © 20 18 by Dennis N. Crouse\nAll rights reserved. This book or any portion thereof\nmay not be reproduced or used in any manner whatsoever\nwithout the express written permission of the publisher and author\nexcept for the use of brief quotations in a book review.\n```\n```\nPrinted in the United States of America\n``

In [35]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embedder)
ids = vector_store.add_documents(documents=documents)

In [37]:
query = "What is the benefits of drinking Silica-rich water"
results = vector_store.similarity_search(query=query,k=3)
print(results[:3])
for doc in results:
    print(f"* {doc.page_content} [{doc.metadata}]")

[Document(id='470', metadata={'source': 'Book'}, page_content='```\nDrinking OSA rich silica water can reduce aluminum accumulation in our bodies\nand prevent 7 terminal diseases in which aluminum is a causal factor: Alzheimer’s ,\nheart disease, stroke, cancer, multiple sclerosis, osteoporosis, and Parkinson’s. Also\ndrinking OSA rich silica water can improve the odds of conception and lower the\nrisk of autism and seizures and even heal from autism and seizures.\n```\nThis technological breakthrough was made by a group of dedicated researchers who have spent\nthe last two decades making the following important discoveries:'), Document(id='200', metadata={'source': 'Book'}, page_content='The advantage of eating vegetables at the expense of meat works best if the vegetables are rich in\nsilica. In longevity regions where drinking water and irrigation water come from the same silica\n\n\nrich sources, there is an enhancement of silica in vegetables by fertilization with silica water.\nI

In [27]:
with open('../Documents/FAQ.md', 'r') as file:
    faq_string = file.read()
print(faq_string)

# Frequently asked questions

## What is silica-rich water?

Silica-rich water contains at least 48 mg/l orthosilicic acid (OSA). All ground water contains
some silica. The amount of silica in ground water depends on the soil composition, and how
long the water is in contact with the soil. The silica in ground water can be low even though
there are high concentrations of other minerals, like calcium and magnesium. Ground water
with at least 48 mg/l OSA is quite rare.

## How much water should I drink?

You want to drink 3 to 4 cups a day (a liter) of silica water. Spread your drinking throughout the
day. If you do an activity where you sweat, drink some silica water before the activity as
aluminum is excreted in your sweat.

You can drink more than 4 cups per day.

You need a water that has at least 48 ppm of OSA. OSA is the form of silica needed to remove
aluminum from the body. OSA is orthosilicic acid.

## How long will it take to see improvements?

Children with Autism generally se

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n##", "\n##", "##"],
    chunk_size=800,
    chunk_overlap=0,
    length_function=len
)
faq_splits = text_splitter.split_text(faq_string)

print(len(faq_splits))
for i in range(len(faq_splits)):
    print(f"* {faq_splits[i]}")

13
* # Frequently asked questions

## What is silica-rich water?

Silica-rich water contains at least 48 mg/l orthosilicic acid (OSA). All ground water contains
some silica. The amount of silica in ground water depends on the soil composition, and how
long the water is in contact with the soil. The silica in ground water can be low even though
there are high concentrations of other minerals, like calcium and magnesium. Ground water
with at least 48 mg/l OSA is quite rare.
* ## How much water should I drink?

You want to drink 3 to 4 cups a day (a liter) of silica water. Spread your drinking throughout the
day. If you do an activity where you sweat, drink some silica water before the activity as
aluminum is excreted in your sweat.

You can drink more than 4 cups per day.

You need a water that has at least 48 ppm of OSA. OSA is the form of silica needed to remove
aluminum from the body. OSA is orthosilicic acid.

## How long will it take to see improvements?

Children with Autism genera